In [ ]:
!pip install indexed-gzip
!pip install networkx
!pip install matplotlib
!pip install pyvis
!pip install plotly
!pip install dash
!pip install numpy
!pip install pickle   

In [141]:
import networkx as nx
import matplotlib.pyplot as plt
from pyvis.network import Network
import plotly.graph_objects as go
from dash import Dash, html, dcc, Input, Output, State
import gzip
import networkx as nx
import numpy as np
import random
import pickle 
import networkx as nx
import dash
from dash import dcc, html, Input, Output, State
import dash.dependencies
import networkx as nx
import plotly.graph_objects as go
import numpy as np
import random
import json
import time
from dash.exceptions import PreventUpdate
import plotly.express as px
from math import sqrt

# Q-Learning parameters
ALPHA = 0.1      # Learning rate
GAMMA = 0.9      # Discount factor
EPSILON = 0.2    # Exploration rate
TRAINING_EPISODES = 1000  # Number of training episodes before gameplay
WIDTH = 5
DEPTH =4

def checkPosVacancy(pos, target_x, target_y):
    for node, (x, y) in pos.items():
        if x == target_x and y == target_y:
            return False
    return True
def manhattanDistance(n1, n2):
    x1=n1[0]; y1 = n1[1]
    x2=n2[0]; y2=n2[1]
    return((abs(x1-x2)+abs(y1-y2)))
def distance(n1, n2):
    x1=n1[0]; y1 = n1[1]
    x2=n2[0]; y2=n2[1]
    return(sqrt((x1-x2)**2+(y1-y2)**2))

In [144]:
def generate_symmetrical_graph(depth=3, width=5):
    G = nx.Graph()
    pos = {}

    G.add_node('Blue')
    G.add_node('Red')
    pos['Blue'] = (0, 0)
    pos['Red'] = (depth * 2 + 2, 0)

    central_nodes = [f'C{i}' for i in range(width)]
    for i, c in enumerate(central_nodes):
        G.add_node(c)
        pos[c] = (depth + 1, i - width // 2)

    def width_for_level(level,width):
        # Decrease density at start, increase towards middle, then medium at end
        if level == 1:
            return (max(2, width // 3),max(2, width // 3))  # small width (e.g., ~1 or 2)
        elif level == depth:
            return (width,width)  # full width at the last level
        else:
            left_levelWidth = random.randrange(2,width)
            right_levelWidth = random.randrange(2,width)
            return (left_levelWidth,right_levelWidth)  # full width at middle layers

    def add_layers(parent_left, parent_right, level):
        if level > depth:
            return
        (left_width, right_width) = width_for_level(level,width)
                                       
        # Construction of the Left Side of the Graph
        for i in range(left_width):
            while True:
                j = random.randrange(-left_width,left_width+1)
                (x,y)=(level, j - left_width // 2)
                if checkPosVacancy(pos,x,y):
                    break
            
            left_name = f'L{level}-{i}'

            G.add_node(left_name)
            if(level == 1):
                G.add_edge(left_name, parent_left)
            pos[left_name] = (x,y)

            if level == depth:
                G.add_edge(left_name, central_nodes[i])
        # Construction of the Right Side of the Graph
        for i in range(right_width):
            while True:
                j = random.randrange(-right_width,right_width+1)
                (x,y)=(depth * 2 + 2 - level, j - right_width // 2)
                
                if checkPosVacancy(pos,x,y):
                    break

            right_name = f'R{level}-{i}'

            G.add_node(right_name)
            if(level == 1):
                G.add_edge(right_name, parent_right)
            pos[right_name] = (x,y)

            if level == depth:
                
                G.add_edge(right_name, central_nodes[i])
            else:
                add_layers(left_name, right_name, level + 1)

    def connectLayers(G, pos, width):
        for i in pos.keys():
            if G.degree[i]>4:
                continue
            for j in pos.keys():
                if G.degree[i]>4:
                    continue
                if (distance(pos[i], pos[j])<=3):
                    G.add_edge(i, j)


        
    add_layers('Blue', 'Red', 1)
    connectLayers(G,pos,width)
    return G, pos

In [145]:
G, pos = generate_symmetrical_graph(6, 5)

# Create state mapping for Q-learning (nodes to indices)
nodes_list = list(G.nodes())
state_to_idx = {node: i for i, node in enumerate(nodes_list)}
idx_to_state = {i: node for i, node in enumerate(nodes_list)}
num_states = len(nodes_list)

# Training analytics storage
class TrainingAnalytics:
    def __init__(self):
        self.reset()
    
    def reset(self):
        self.blue_wins = []
        self.red_wins = []
        self.exploration_rates = []
        self.exploitation_rates = []
        self.episode_lengths = []
        self.win_rate_window = 50  # Calculate win rate over last 50 games
        
    def log_episode(self, winner, episode_length, blue_explorations, blue_exploitations, red_explorations, red_exploitations):
        # Log winner
        self.blue_wins.append(1 if winner == "Blue" else 0)
        self.red_wins.append(1 if winner == "Red" else 0)
        
        # Log episode length
        self.episode_lengths.append(episode_length)
        
        # Log exploration vs exploitation ratios
        total_blue_moves = blue_explorations + blue_exploitations
        total_red_moves = red_explorations + red_exploitations
        
        blue_exploration_rate = blue_explorations / total_blue_moves if total_blue_moves > 0 else 0
        red_exploration_rate = red_explorations / total_red_moves if total_red_moves > 0 else 0
        
        # Store average exploration rate for this episode
        avg_exploration_rate = (blue_exploration_rate + red_exploration_rate) / 2
        self.exploration_rates.append(avg_exploration_rate)
        self.exploitation_rates.append(1 - avg_exploration_rate)
    
    def get_win_rates(self):
        if len(self.blue_wins) == 0:
            return [], []
        
        blue_win_rates = []
        red_win_rates = []
        
        for i in range(len(self.blue_wins)):
            start_idx = max(0, i - self.win_rate_window + 1)
            end_idx = i + 1
            
            blue_wins_in_window = sum(self.blue_wins[start_idx:end_idx])
            red_wins_in_window = sum(self.red_wins[start_idx:end_idx])
            total_games_in_window = end_idx - start_idx
            
            blue_win_rate = blue_wins_in_window / total_games_in_window
            red_win_rate = red_wins_in_window / total_games_in_window
            
            blue_win_rates.append(blue_win_rate)
            red_win_rates.append(red_win_rate)
        
        return blue_win_rates, red_win_rates

# Global analytics instance
training_analytics = TrainingAnalytics()

class QLearningBot:
    def __init__(self, name, start_node, target_node, num_states):
        self.name = name
        self.start_node = start_node
        self.target_node = target_node
        self.q_table = {}  # Using a dictionary for efficient sparse representation
        self.current_state = start_node
        self.path = [start_node]
        self.opponent_path = []
        
        # Analytics tracking
        self.explorations = 0
        self.exploitations = 0
        
    def get_valid_actions(self, state):
        return list(G.neighbors(state))
    
    def choose_action(self, state, valid_actions, epsilon):
        if random.random() < epsilon:  # Exploration
            self.explorations += 1
            return random.choice(valid_actions)
        else:  # Exploitation
            self.exploitations += 1
            q_values = [self.get_q_value(state, action) for action in valid_actions]
            max_q = max(q_values)
            # Handle multiple actions with the same max Q-value
            max_actions = [valid_actions[i] for i, q in enumerate(q_values) if q == max_q]
            return random.choice(max_actions)
    
    def get_q_value(self, state, action):
        return self.q_table.get((state, action), 0.0)
    
    def update_q_value(self, state, action, reward, next_state, alpha, gamma):
        # Select best next action based on Q-values
        next_actions = self.get_valid_actions(next_state)
        if next_actions:
            next_q_values = [self.get_q_value(next_state, next_action) for next_action in next_actions]
            max_next_q = max(next_q_values)
        else:
            max_next_q = 0
        
        # Q-Learning update formula
        old_q = self.get_q_value(state, action)
        new_q = old_q + alpha * (reward + gamma * max_next_q - old_q)
        self.q_table[(state, action)] = new_q
    
    def reset(self):
        self.current_state = self.start_node
        self.path = [self.start_node]
        # Reset analytics for new episode
        self.explorations = 0
        self.exploitations = 0
    
    def calculate_reward(self, next_state, opponent_path):
        # Base rewards
        if next_state == self.target_node:
            return 100  # Reached target
        
        # Penalty for revisiting nodes
        if next_state in self.path:
            return -5
            
        # Penalty for moving to opponent-occupied nodes
        if next_state in opponent_path:
            return -10
            
        # Encourage moving towards target
        shortest_path_length = nx.shortest_path_length(G, next_state, self.target_node)
        path_reward = 5 / (shortest_path_length + 1)  # Smaller distance = higher reward
            
        return path_reward
        
    def make_move(self, opponent_path, training=False):
        self.opponent_path = opponent_path
        valid_actions = self.get_valid_actions(self.current_state)
        
        # Filter out actions that lead to already visited nodes (unless training)
        if not training:
            valid_actions = [a for a in valid_actions if a not in self.path]
            if not valid_actions:  # If stuck, allow revisiting
                valid_actions = self.get_valid_actions(self.current_state)
        
        # If no valid actions, stay in place
        if not valid_actions:
            return self.current_state
            
        # Choose next action based on Q-learning policy
        epsilon = EPSILON if training else 0.1  # Less exploration during actual gameplay
        next_state = self.choose_action(self.current_state, valid_actions, epsilon)
        
        # Calculate reward
        reward = self.calculate_reward(next_state, opponent_path)
        
        # Update Q-table if training
        if training:
            self.update_q_value(self.current_state, next_state, reward, next_state, ALPHA, GAMMA)
            
        # Update current state and path
        self.current_state = next_state
        self.path.append(next_state)
        
        return next_state

# Create the bots
blue_bot = QLearningBot("Blue", "Blue", "Red", num_states)
red_bot = QLearningBot("Red", "Red", "Blue", num_states)

# Training phase
def train_bots():
    print("Training bots...")
    training_analytics.reset()
    
    for episode in range(TRAINING_EPISODES):
        blue_bot.reset()
        red_bot.reset()
        
        # Simulate a game until one bot reaches target or max steps
        max_steps = 50
        winner = None
        
        for step in range(max_steps):
            blue_move = blue_bot.make_move(red_bot.path, training=True)
            if blue_move == blue_bot.target_node:
                winner = "Blue"
                break
                
            red_move = red_bot.make_move(blue_bot.path, training=True)
            if red_move == red_bot.target_node:
                winner = "Red"
                break
        
        # If no winner after max steps, declare a draw (count as no win for both)
        if winner is None:
            winner = "Draw"
        
        # Log training analytics
        episode_length = len(blue_bot.path) + len(red_bot.path) - 2  # Subtract starting positions
        training_analytics.log_episode(
            winner, 
            episode_length,
            blue_bot.explorations,
            blue_bot.exploitations,
            red_bot.explorations,
            red_bot.exploitations
        )
        
        # Print progress every 100 episodes
        if (episode + 1) % 100 == 0:
            blue_win_rates, red_win_rates = training_analytics.get_win_rates()
            current_blue_wr = blue_win_rates[-1] if blue_win_rates else 0
            current_red_wr = red_win_rates[-1] if red_win_rates else 0
            print(f"Episode {episode + 1}/{TRAINING_EPISODES} - Blue WR: {current_blue_wr:.2%}, Red WR: {current_red_wr:.2%}")
    
    print("Training completed!")

# Train the bots before starting the app
train_bots()

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Graph War Game with Q-Learning Bots", style={'textAlign': 'center', 'color': 'white'}),
    html.Div([
        html.Button("Start Game", id="start-game", n_clicks=0, 
                   style={'fontSize': '16px', 'padding': '10px 20px', 'margin': '5px', 'backgroundColor': '#4CAF50', 'color': 'white', 'border': 'none', 'borderRadius': '4px'}),
        html.Button("Reset Game", id="reset-game", n_clicks=0,
                   style={'fontSize': '16px', 'padding': '10px 20px', 'margin': '5px', 'backgroundColor': '#f44336', 'color': 'white', 'border': 'none', 'borderRadius': '4px'}),
        html.Button("Play as Blue", id="play-as-blue", n_clicks=0,
                   style={'fontSize': '16px', 'padding': '10px 20px', 'margin': '5px', 'backgroundColor': '#2196F3', 'color': 'white', 'border': 'none', 'borderRadius': '4px'}),
        html.Button("Show Training Analytics", id="show-analytics", n_clicks=0,
                   style={'fontSize': '16px', 'padding': '10px 20px', 'margin': '5px', 'backgroundColor': '#FF9800', 'color': 'white', 'border': 'none', 'borderRadius': '4px'}),
        html.Div([
            html.Label("War Time Scale(ms): ", style={'color': 'white', 'marginRight': '10px', 'fontSize': '16px'}),
            dcc.Slider(
                id='speed-slider',
                min=100,
                max=2000,
                step=100,
                value=500,
                marks={
                    100: {'label': 'Fast', 'style': {'color': 'white'}},
                    500: {'label': '500ms', 'style': {'color': 'white'}},
                    1000: {'label': '1s', 'style': {'color': 'white'}},
                    2000: {'label': 'Slow', 'style': {'color': 'white'}}
                }
            )
        ], style={'width': '50%', 'margin': '0 auto', 'padding': '10px'})
    ], style={'textAlign': 'center', 'margin': '10px'}),
    
    # Training Analytics Section (initially hidden)
    html.Div([
        html.H2("Training Analytics", style={'textAlign': 'center', 'color': 'white', 'marginBottom': '20px'}),
        html.Div([
            html.Div([
                dcc.Graph(id='winrate-graph')
            ], style={'width': '50%', 'display': 'inline-block', 'verticalAlign': 'top'}),
            html.Div([
                dcc.Graph(id='exploration-graph')
            ], style={'width': '50%', 'display': 'inline-block', 'verticalAlign': 'top'})
        ]),
        html.Div([
            html.H3("Training Summary", style={'color': 'white', 'textAlign': 'center'}),
            html.Div(id='training-summary', style={'color': 'white', 'textAlign': 'center', 'fontSize': '14px'})
        ], style={'marginTop': '20px', 'padding': '10px', 'backgroundColor': 'rgba(0, 0, 0, 0.3)', 'borderRadius': '10px'})
    ], id='analytics-section', style={'display': 'none', 'backgroundColor': 'rgba(0, 0, 0, 0.2)', 'padding': '20px', 'borderRadius': '10px', 'margin': '20px 0'}),
    
    dcc.Graph(id='graph'),
    
    # Scoreboard section
    html.Div([
        html.H2("Scoreboard", style={'textAlign': 'center', 'color': 'white', 'marginBottom': '10px'}),
        html.Div([
            html.Div([
                html.H3("Blue Score", style={'color': '#2196F3', 'marginBottom': '5px'}),
                html.Div(id='blue-score', style={'fontSize': '32px', 'fontWeight': 'bold', 'color': '#2196F3'})
            ], style={'width': '45%', 'display': 'inline-block', 'textAlign': 'center', 'backgroundColor': 'rgba(33, 150, 243, 0.1)', 'padding': '10px', 'borderRadius': '5px'}),
            html.Div([
                html.H3("Red Score", style={'color': '#f44336', 'marginBottom': '5px'}),
                html.Div(id='red-score', style={'fontSize': '32px', 'fontWeight': 'bold', 'color': '#f44336'})
            ], style={'width': '45%', 'display': 'inline-block', 'textAlign': 'center', 'backgroundColor': 'rgba(244, 67, 54, 0.1)', 'padding': '10px', 'borderRadius': '5px'})
        ], style={'display': 'flex', 'justifyContent': 'space-around', 'marginBottom': '20px'})
    ], style={'backgroundColor': 'rgba(0, 0, 0, 0.3)', 'padding': '15px', 'borderRadius': '10px', 'margin': '20px 0'}),
    
    html.Div([
        html.Div(id='blue-status', style={'width': '45%', 'display': 'inline-block', 'textAlign': 'center'}),
        html.Div(id='red-status', style={'width': '45%', 'display': 'inline-block', 'textAlign': 'center'}),
    ]),
    html.Div(id='game-status', style={'textAlign': 'center', 'fontWeight': 'bold', 'fontSize': '20px', 'margin': '10px'}),
    dcc.Store(id='game-state', data={
        'blue_path': ["Blue"],
        'red_path': ["Red"],
        'game_active': False,
        'winner': None,
        'turn': 'blue',
        'player_controlled': False,  # Flag to indicate if Blue is player-controlled
        'waiting_for_player_move': False,  # Flag to indicate if we're waiting for player to make a move
        'blue_score': 0,  # Initialize Blue's score
        'red_score': 0,   # Initialize Red's score
        'contested_nodes': []  # Track contested nodes
    }),
    dcc.Interval(id='game-interval', interval=500, disabled=True, n_intervals=0),  # Default 500ms interval
], style={'backgroundColor': '#333', 'padding': '20px', 'fontFamily': 'Arial, sans-serif', 'color': 'white'})

# Callback to toggle analytics section visibility
@app.callback(
    Output('analytics-section', 'style'),
    Input('show-analytics', 'n_clicks'),
    State('analytics-section', 'style'),
    prevent_initial_call=True
)
def toggle_analytics(n_clicks, current_style):
    if n_clicks > 0:
        # Toggle visibility
        if current_style.get('display') == 'none':
            return {'display': 'block', 'backgroundColor': 'rgba(0, 0, 0, 0.2)', 'padding': '20px', 'borderRadius': '10px', 'margin': '20px 0'}
        else:
            return {'display': 'none'}
    return current_style

# Callback to update analytics graphs
@app.callback(
    Output('winrate-graph', 'figure'),
    Output('exploration-graph', 'figure'),
    Output('training-summary', 'children'),
    Input('show-analytics', 'n_clicks'),
    prevent_initial_call=True
)
def update_analytics_graphs(n_clicks):
    if n_clicks == 0:
        raise PreventUpdate
    
    # Get win rates
    blue_win_rates, red_win_rates = training_analytics.get_win_rates()
    episodes = list(range(1, len(blue_win_rates) + 1))
    
    # Create win rate graph
    winrate_fig = go.Figure()
    winrate_fig.add_trace(go.Scatter(
        x=episodes, y=[wr * 100 for wr in blue_win_rates],
        mode='lines', name='Blue Win Rate',
        line=dict(color='blue', width=2)
    ))
    winrate_fig.add_trace(go.Scatter(
        x=episodes, y=[wr * 100 for wr in red_win_rates],
        mode='lines', name='Red Win Rate',
        line=dict(color='red', width=2)
    ))
    
    winrate_fig.update_layout(
        title="Win Rate Over Training Episodes",
        xaxis_title="Episode",
        yaxis_title="Win Rate (%)",
        plot_bgcolor='rgba(0,0,0,0.1)',
        paper_bgcolor='rgba(0,0,0,0)',
        font=dict(color='white'),
        legend=dict(bgcolor='rgba(0,0,0,0.5)')
    )
    
    # Create exploration vs exploitation graph
    episodes_full = list(range(1, len(training_analytics.exploration_rates) + 1))
    
    exploration_fig = go.Figure()
    exploration_fig.add_trace(go.Scatter(
        x=episodes_full, y=[er * 100 for er in training_analytics.exploration_rates],
        mode='lines', name='Exploration Rate',
        line=dict(color='orange', width=2)
    ))
    exploration_fig.add_trace(go.Scatter(
        x=episodes_full, y=[er * 100 for er in training_analytics.exploitation_rates],
        mode='lines', name='Exploitation Rate',
        line=dict(color='green', width=2)
    ))
    
    exploration_fig.update_layout(
        title="Exploration vs Exploitation Over Training",
        xaxis_title="Episode",
        yaxis_title="Rate (%)",
        plot_bgcolor='rgba(0,0,0,0.1)',
        paper_bgcolor='rgba(0,0,0,0)',
        font=dict(color='white'),
        legend=dict(bgcolor='rgba(0,0,0,0.5)')
    )
    
    # Create training summary
    if len(blue_win_rates) > 0:
        final_blue_wr = blue_win_rates[-1]
        final_red_wr = red_win_rates[-1]
        avg_episode_length = np.mean(training_analytics.episode_lengths)
        avg_exploration_rate = np.mean(training_analytics.exploration_rates)
        
        summary = html.Div([
            html.P(f"Total Training Episodes: {TRAINING_EPISODES}"),
            html.P(f"Final Blue Win Rate: {final_blue_wr:.1%}"),
            html.P(f"Final Red Win Rate: {final_red_wr:.1%}"),
            html.P(f"Average Episode Length: {avg_episode_length:.1f} moves"),
            html.P(f"Average Exploration Rate: {avg_exploration_rate:.1%}"),
            html.P(f"Q-Table Size - Blue: {len(blue_bot.q_table)}, Red: {len(red_bot.q_table)}")
        ])
    else:
        summary = html.P("No training data available")
    
    return winrate_fig, exploration_fig, summary

def make_edge_trace():
    edge_x = []
    edge_y = []
    for edge in G.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x += [x0, x1, None]
        edge_y += [y0, y1, None]
    return go.Scatter(
        x=edge_x, y=edge_y, 
        line=dict(width=1, color='#000'),
        hoverinfo='none', 
        mode='lines'
    )

def make_node_trace(node_colors, highlight_valid_moves=None):
    node_x = []
    node_y = []
    text = []
    node_sizes = []
    for node in G.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)
        text.append(node)
        # Highlight valid moves for player by making nodes bigger
        if highlight_valid_moves and node in highlight_valid_moves:
            node_sizes.append(30)  # Bigger size for valid moves
        else:
            node_sizes.append(20)  # Default size
            
    return go.Scatter(
        x=node_x, y=node_y,
        mode='markers+text',
        text=text,
        textposition='bottom center',
        hoverinfo='text',
        marker=dict(color=node_colors, size=node_sizes, line_width=2)
    )

# Get valid moves for the current position
def get_valid_moves(current_node, visited_nodes):
    valid_moves = list(G.neighbors(current_node))
    # Filter out already visited nodes
    valid_moves = [node for node in valid_moves if node not in visited_nodes]
    if not valid_moves:  # If stuck, allow revisiting
        valid_moves = list(G.neighbors(current_node))
    return valid_moves

# Function to update scores based on new moves
def update_scores(blue_path, red_path, old_blue_path, old_red_path, contested_nodes, blue_score, red_score):
    # Check for new nodes in blue path
    if len(blue_path) > len(old_blue_path):
        new_blue_node = blue_path[-1]
        
        # Check if blue captured red's base
        if new_blue_node == "Red":
            blue_score += 10
        # Check if node is contested
        elif new_blue_node in red_path:
            blue_score -= 2  # Blue is attacker
            red_score -= 1   # Red is defender
            # Add to contested nodes if not already there
            if new_blue_node not in contested_nodes:
                contested_nodes.append(new_blue_node)
        # Regular node collection penalty
        else:
            blue_score -= 1
    
    # Check for new nodes in red path
    if len(red_path) > len(old_red_path):
        new_red_node = red_path[-1]
        
        # Check if red captured blue's base
        if new_red_node == "Blue":
            red_score += 10
        # Check if node is contested
        elif new_red_node in blue_path:
            red_score -= 2   # Red is attacker
            blue_score -= 1  # Blue is defender
            # Add to contested nodes if not already there
            if new_red_node not in contested_nodes:
                contested_nodes.append(new_red_node)
        # Regular node collection penalty
        else:
            red_score -= 1
    
    return blue_score, red_score, contested_nodes

@app.callback(
    Output('game-interval', 'disabled'),
    Output('game-interval', 'interval'),
    Input('start-game', 'n_clicks'),
    Input('reset-game', 'n_clicks'),
    Input('play-as-blue', 'n_clicks'),
    Input('game-state', 'data'),
    Input('speed-slider', 'value'),
    prevent_initial_call=True
)
def toggle_game_interval(start_clicks, reset_clicks, play_as_blue_clicks, game_state, speed_value):
    ctx = dash.callback_context
    if not ctx.triggered:
        return True, speed_value
    
    trigger_id = ctx.triggered[0]['prop_id'].split('.')[0]
    
    if trigger_id == 'start-game' and start_clicks > 0:
        # Enable the interval for AI vs AI mode
        return False, speed_value  # Enable interval with selected speed
    elif trigger_id == 'play-as-blue' and play_as_blue_clicks > 0:
        # For player mode, only enable interval when it's red's turn
        if game_state.get('turn') == 'red' and game_state.get('game_active', False):
            return False, speed_value
        else:
            return True, speed_value  # Disable interval, waiting for player click
    elif trigger_id == 'reset-game':
        return True, speed_value   # Disable interval
    elif trigger_id == 'game-state':
        # Handle game state changes
        if game_state.get('winner'):
            return True, speed_value  # Disable interval if game is over
        
        # In player mode, enable/disable based on whose turn it is
        if game_state.get('player_controlled', False):
            if game_state.get('turn') == 'red' and game_state.get('game_active', False):
                return False, speed_value  # Enable for Red's turn
            else:
                return True, speed_value   # Disable for Blue's turn (player controlled)
    elif trigger_id == 'speed-slider':
        # Only update speed, keep disabled state unchanged
        currently_disabled = dash.no_update
        if 'game-interval.disabled' in ctx.inputs_list[0]:
            currently_disabled = ctx.inputs_list[0]['game-interval.disabled']
        return currently_disabled, speed_value
        
    return dash.no_update, speed_value

@app.callback(
    Output('game-state', 'data'),
    Input('game-interval', 'n_intervals'),
    Input('start-game', 'n_clicks'),
    Input('reset-game', 'n_clicks'),
    Input('play-as-blue', 'n_clicks'),
    Input('graph', 'clickData'),
    State('game-state', 'data'),
    prevent_initial_call=True
)
def update_game_state(n_intervals, start_clicks, reset_clicks, play_as_blue_clicks, click_data, game_state):
    ctx = dash.callback_context
    if not ctx.triggered:
        return game_state
        
    trigger_id = ctx.triggered[0]['prop_id'].split('.')[0]
    
    # Start game (AI vs AI mode)
    if trigger_id == 'start-game' and start_clicks > 0:
        # Only reset the game state if the game isn't already active
        if not game_state['game_active']:
            blue_bot.reset()
            red_bot.reset()
            return {
                'blue_path': ["Blue"],
                'red_path': ["Red"],
                'game_active': True,
                'winner': None,
                'turn': 'blue',
                'player_controlled': False,
                'waiting_for_player_move': False,
                'blue_score': 0,
                'red_score': 0,
                'contested_nodes': []
            }
        return game_state  # Don't modify if already active
    
    # Play as Blue mode
    if trigger_id == 'play-as-blue' and play_as_blue_clicks > 0:
        blue_bot.reset()
        red_bot.reset()
        return {
            'blue_path': ["Blue"],
            'red_path': ["Red"],
            'game_active': True,
            'winner': None,
            'turn': 'blue',
            'player_controlled': True,
            'waiting_for_player_move': True,  # Waiting for player's first move
            'blue_score': 0,
            'red_score': 0,
            'contested_nodes': []
        }
    
    # Reset game
    if trigger_id == 'reset-game' and reset_clicks > 0:
        blue_bot.reset()
        red_bot.reset()
        return {
            'blue_path': ["Blue"],
            'red_path': ["Red"],
            'game_active': False,
            'winner': None,
            'turn': 'blue',
            'player_controlled': False,
            'waiting_for_player_move': False,
            'blue_score': 0,
            'red_score': 0,
            'contested_nodes': []
        }
    
    # Handle player clicks on nodes (only when it's Blue's turn and player controlled)
    if trigger_id == 'graph' and click_data and game_state['player_controlled'] and game_state['turn'] == 'blue' and game_state['waiting_for_player_move']:
        # Get the node that was clicked
        clicked_node = click_data['points'][0]['text']
        blue_path = game_state['blue_path'].copy()
        red_path = game_state['red_path'].copy()
        old_blue_path = blue_path.copy()
        current_node = blue_path[-1]
        
        # Check if the clicked node is a valid move
        valid_moves = get_valid_moves(current_node, blue_path)
        
        if clicked_node in valid_moves:
            # Valid move, update Blue's path
            blue_path.append(clicked_node)
            
            # Update scores
            blue_score, red_score, contested_nodes = update_scores(
                blue_path, red_path, old_blue_path, red_path, 
                game_state['contested_nodes'], game_state['blue_score'], game_state['red_score']
            )
            
            # Check if Blue reached target
            winner = None
            if clicked_node == "Red":
                winner = "Blue"
                
            # Change turn to Red
            return {
                'blue_path': blue_path,
                'red_path': red_path,
                'game_active': True,
                'winner': winner,
                'turn': 'red',
                'player_controlled': True,
                'waiting_for_player_move': False,  # No longer waiting for player input
                'blue_score': blue_score,
                'red_score': red_score,
                'contested_nodes': contested_nodes
            }
        else:
            # Invalid move, don't update
            return game_state
    
    # If game already has a winner or is not active, don't update
    if game_state['winner'] or not game_state['game_active']:
        return game_state
        
    # Bot moves - this runs automatically with each interval tick when game is active
    if trigger_id == 'game-interval':
        blue_path = game_state['blue_path'].copy()
        red_path = game_state['red_path'].copy()
        old_blue_path = blue_path.copy()
        old_red_path = red_path.copy()
        turn = game_state['turn']
        winner = None
        player_controlled = game_state['player_controlled']
        blue_score = game_state['blue_score']
        red_score = game_state['red_score']
        contested_nodes = game_state['contested_nodes']
        
        # Blue's turn (AI controlled - only in AI vs AI mode)
        if turn == 'blue' and not player_controlled:
            blue_bot.path = blue_path
            blue_bot.current_state = blue_path[-1]
            next_move = blue_bot.make_move(red_path)
            blue_path.append(next_move)
            
            # Update scores
            blue_score, red_score, contested_nodes = update_scores(
                blue_path, red_path, old_blue_path, old_red_path, 
                contested_nodes, blue_score, red_score
            )
            
            # Check if Blue reached target
            if next_move == "Red":
                winner = "Blue"
            
            turn = 'red'
        
        # Red's turn (always AI controlled)
        elif turn == 'red':
            red_bot.path = red_path
            red_bot.current_state = red_path[-1]
            next_move = red_bot.make_move(blue_path)
            red_path.append(next_move)
            
            # Update scores
            blue_score, red_score, contested_nodes = update_scores(
                blue_path, red_path, old_blue_path, old_red_path, 
                contested_nodes, blue_score, red_score
            )
            
            # Check if Red reached target
            if next_move == "Blue":
                winner = "Red"
            
            turn = 'blue'
            # If in player mode, we're now waiting for player input
            waiting_for_player_move = player_controlled
            
            return {
                'blue_path': blue_path,
                'red_path': red_path,
                'game_active': True,
                'winner': winner,
                'turn': turn,
                'player_controlled': player_controlled,
                'waiting_for_player_move': waiting_for_player_move,
                'blue_score': blue_score,
                'red_score': red_score,
                'contested_nodes': contested_nodes
            }
            
        return {
            'blue_path': blue_path,
            'red_path': red_path,
            'game_active': True,
            'winner': winner,
            'turn': turn,
            'player_controlled': player_controlled,
            'waiting_for_player_move': game_state['waiting_for_player_move'],
            'blue_score': blue_score,
            'red_score': red_score,
            'contested_nodes': contested_nodes
        }
    
    return game_state

@app.callback(
    Output('graph', 'figure'),
    Output('blue-status', 'children'),
    Output('red-status', 'children'),
    Output('game-status', 'children'),
    Output('blue-score', 'children'),
    Output('red-score', 'children'),
    Input('game-state', 'data'),
    prevent_initial_call=True
)
def update_graph(game_state):
    blue_path = game_state['blue_path']
    red_path = game_state['red_path']
    winner = game_state['winner']
    blue_score = game_state['blue_score']
    red_score = game_state['red_score']
    contested_nodes = game_state.get('contested_nodes', [])
    
    # Node colors
    node_colors = []
    for node in G.nodes():
        if node == "Blue" and node in red_path:
            node_colors.append('purple')  # Blue node captured by Red
        elif node == "Red" and node in blue_path:
            node_colors.append('purple')  # Red node captured by Blue
        elif node in blue_path and node in red_path:
            node_colors.append('purple')  # Contested node
        elif node in blue_path:
            node_colors.append('blue')
        elif node in red_path:
            node_colors.append('red')
        else:
            node_colors.append('grey')
    
    # Highlight valid moves for player in player mode
    highlight_valid_moves = None
    if game_state.get('player_controlled', False) and game_state.get('waiting_for_player_move', False):
        current_blue_pos = blue_path[-1]
        highlight_valid_moves = get_valid_moves(current_blue_pos, blue_path)
    
    # Create the graph
    edge_trace = make_edge_trace()
    node_trace = make_node_trace(node_colors, highlight_valid_moves)
    fig = go.Figure(
        data=[edge_trace, node_trace],
        layout=go.Layout(
            showlegend=False,
            margin=dict(b=20, l=5, r=5, t=40),
            hovermode='closest',
            xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
            yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
            clickmode='event+select',
            title="War Game",
            plot_bgcolor='rgba(0,0,0,0)',
            paper_bgcolor='rgba(0,0,0,0)',
            font=dict(color='white'),
            images=[
                dict(
                    source="https://foxholehq.net/assets/imgs/worldmap.png",  # Replace with your image URL or base64
                    xref="paper",
                    yref="paper",
                    x=0.05,
                    y=1.5,
                    sizex=0.9,
                    sizey=1.8,
                    sizing="stretch",
                    opacity=0.8,
                    layer="below"
                )
            ]
        )
    )
    
    # Add path lines for each bot
    if len(blue_path) > 1:
        blue_x, blue_y = [], []
        for node in blue_path:
            x, y = pos[node]
            blue_x.append(x)
            blue_y.append(y)
        fig.add_trace(go.Scatter(
            x=blue_x, y=blue_y,
            line=dict(width=2, color='rgba(0, 0, 255, 0.7)'),
            hoverinfo='none',
            mode='lines',
            name='Blue Path'
        ))
    
    if len(red_path) > 1:
        red_x, red_y = [], []
        for node in red_path:
            x, y = pos[node]
            red_x.append(x)
            red_y.append(y)
        fig.add_trace(go.Scatter(
            x=red_x, y=red_y,
            line=dict(width=2, color='rgba(255, 0, 0, 0.7)'),
            hoverinfo='none',
            mode='lines',
            name='Red Path'
        ))
    
    # Status updates
    moves_count = len(blue_path) + len(red_path) - 2  # Subtract the 2 starting positions
    
    # Add player control info to Blue status if applicable
    player_status = ""
    if game_state.get('player_controlled', False) and game_state.get('turn') == 'blue' and not winner:
        player_status = html.P("👉 Click on a highlighted node to move", style={"color": "yellow", "fontWeight": "bold"})
    
    # Scoreboard details
    contested_nodes_info = ""
    if contested_nodes:
        contested_nodes_info = html.P(f"Contested nodes: {', '.join(contested_nodes)}", style={"color": "purple"})
    
    blue_status = html.Div([
        html.H3("Blue Team" + (" (You)" if game_state.get('player_controlled', False) else ""), 
               style={"color": "blue"}),
        html.P(f"Current position: {blue_path[-1]}", style={"color": "white"}),
        html.P(f"Path: {' → '.join(blue_path[-5:] if len(blue_path) > 5 else blue_path)}", style={"color": "white"}),
        html.P(f"Nodes captured: {len(blue_path) - 1}", style={"color": "lightblue"}),
        player_status
    ])
    
    red_status = html.Div([
        html.H3("Red Team", style={"color": "red"}),
        html.P(f"Current position: {red_path[-1]}", style={"color": "white"}),
        html.P(f"Path: {' → '.join(red_path[-5:] if len(red_path) > 5 else red_path)}", style={"color": "white"}),
        html.P(f"Nodes captured: {len(red_path) - 1}", style={"color": "lightpink"})
    ])
    
    # Game status display with scoring explanation
    if winner:
        moves_text = f"Game finished in {moves_count} moves!"
        game_status = html.Div([
            html.H2(f"{winner} Team Wins!", style={"color": "green"}),
            html.P(moves_text, style={"color": "white"}),
            contested_nodes_info,
            html.Div([
                html.H4("Scoring System:", style={"color": "gold", "marginTop": "10px", "marginBottom": "5px"}),
                html.Ul([
                    html.Li("Each node captured: -1 point", style={"color": "white"}),
                    html.Li("Enemy base captured: +10 points", style={"color": "white"}),
                    html.Li("Contesting a node: -2 for attacker, -1 for defender", style={"color": "white"})
                ], style={"textAlign": "left", "paddingLeft": "20px"})
            ])
        ])
    elif game_state['game_active']:
        mode_text = "Player vs Bot" if game_state.get('player_controlled', False) else "Bot vs Bot"
        game_status = html.Div([
            html.H3(f"Current Turn: {game_state['turn'].capitalize()} Team", 
                  style={"color": "blue" if game_state['turn'] == 'blue' else "red"}),
            html.P(f"Mode: {mode_text}", style={"color": "white"}),
            html.P(f"Moves made: {moves_count}", style={"color": "white"}),
            contested_nodes_info,
            html.Div([
                html.H4("Scoring System:", style={"color": "gold", "marginTop": "10px", "marginBottom": "5px"}),
                html.Ul([
                    html.Li("Each node captured: -1 point", style={"color": "white"}),
                    html.Li("Enemy base captured: +10 points", style={"color": "white"}),
                    html.Li("Contesting a node: -2 for attacker, -1 for defender", style={"color": "white"})
                ], style={"textAlign": "left", "paddingLeft": "20px"})
            ])
        ])
    else:
        game_status = html.Div([
            html.H3("Press 'Start Game' for Bot vs Bot mode", style={"color": "white"}),
            html.H3("Press 'Play as Blue' to control the Blue team", style={"color": "white"}),
            html.Div([
                html.H4("Scoring System:", style={"color": "gold", "marginTop": "10px", "marginBottom": "5px"}),
                html.Ul([
                    html.Li("Each node captured: -1 point", style={"color": "white"}),
                    html.Li("Enemy base captured: +10 points", style={"color": "white"}),
                    html.Li("Contesting a node: -2 for attacker, -1 for defender", style={"color": "white"})
                ], style={"textAlign": "left", "paddingLeft": "20px"})
            ])
        ])


    # Format scores with "+" prefix for positive scores
    blue_score_text = f"+{blue_score}" if blue_score > 0 else str(blue_score)
    red_score_text = f"+{red_score}" if red_score > 0 else str(red_score)
    
    return fig, blue_status, red_status, game_status, blue_score_text, red_score_text

if __name__ == '__main__':
    app.run(debug=True)

Training bots...
Episode 100/1000 - Blue WR: 62.00%, Red WR: 38.00%
Episode 200/1000 - Blue WR: 52.00%, Red WR: 48.00%
Episode 300/1000 - Blue WR: 70.00%, Red WR: 30.00%
Episode 400/1000 - Blue WR: 64.00%, Red WR: 36.00%
Episode 500/1000 - Blue WR: 56.00%, Red WR: 44.00%
Episode 600/1000 - Blue WR: 60.00%, Red WR: 40.00%
Episode 700/1000 - Blue WR: 50.00%, Red WR: 50.00%
Episode 800/1000 - Blue WR: 64.00%, Red WR: 36.00%
Episode 900/1000 - Blue WR: 50.00%, Red WR: 50.00%
Episode 1000/1000 - Blue WR: 60.00%, Red WR: 40.00%
Training completed!
